In [1]:
import pandas as pd 
import numpy as np

In [2]:

data=pd.read_csv(r"C:\Users\CG-DTE\Desktop\AI\GITHUB\Predicting bankruptcy\low level model analysis\1year.csv")

### above feature set i have gone through some feature engineering steps
#### missing imputation------>gaussian transformation ----->decision tree descritisation------>scaling --->feature selection
#### [['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55']] i got these features

### but important analysis i found that feature set using these above feature engineering steps giving best result on original data aftter imputation only

In [3]:
year_1=data[['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55','class']]

In [4]:
k1=year_1.columns.tolist()

In [5]:
#replacing each ? to NAN
for i in k1:
    year_1[i] = year_1[i].replace('?' , np.nan)

<ipython-input-5-399612e69be0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_1[i] = year_1[i].replace('?' , np.nan)


In [6]:
#converting object data into numericals(float64)
year_1=year_1.apply(lambda col:pd.to_numeric(col,errors='coerce'))

In [7]:
year_1

,Attr5,Attr11,Attr21,Attr22,Attr24,Attr27,Attr33,Attr40,Attr46,Attr55,class
0,32.3510,0.249760,1.24790,0.214020,0.477060,1.45820,3.8772,0.662950,1.52250,348690.0,0
1,14.7860,0.261140,1.42930,0.248060,NaN,88.44400,2.9876,0.086422,1.12520,2304.6,0
2,-1.1523,0.312580,1.42830,0.302600,NaN,86.01100,2.0630,0.322020,1.01010,6332.7,0
3,51.9520,0.092704,1.50690,0.115500,0.171930,0.94076,3.9948,0.401390,1.56960,20545.0,0
4,-7.3128,0.187320,NaN,0.198320,0.187320,1.41380,2.4823,0.293040,0.95787,3186.6,0
...,...,...,...,...,...,...,...,...,...,...,...
7022,-18.4490,0.018371,1.20480,0.032032,0.018371,NaN,11.5340,0.302270,0.44959,-2732.8,1
7023,-42.2320,-0.015036,1.17170,-0.008746,-0.015036,NaN,3.9222,0.022246,0.56298,-2557.1,1
7024,9.7417,0.012022,NaN,0.014559,0.012022,NaN,7.0952,0.012816,1.19030,4826.9,1
7025,-121.9200,0.006414,0.64405,0.006414,0.045607,0.14843,1.7183,0.096108,0.13122,-1633.0,1


### so here i m building model on entire dataset for , so i m not divinding into train and test dataset , so that i do have ample amount of data to train our model more accurately

In [8]:
predictors=['Attr5', 'Attr11', 'Attr21', 'Attr22', 'Attr24', 'Attr27', 'Attr33', 'Attr40', 'Attr46', 'Attr55']
target=['class']
X=year_1[predictors]
y=year_1[target]

In [9]:
#X
#y

In [10]:
from sklearn.impute import SimpleImputer

In [11]:
# Now we impute the missing values with SimpleImputer

# create an instance of the simple imputer
# we indicate that we want to impute with the median
mean_imputer = SimpleImputer(strategy='mean',)

# we fit and transform the predictors dataframe
# the imputer will learn the mean of all variables
X_new=mean_imputer.fit_transform(X[predictors])

In [12]:
y_1=y.copy()
X_1=pd.DataFrame(X_new, columns=predictors)

### over sampling through SMOTE is performed

In [13]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2) 
X_res, y_res = sm.fit_resample(X_1, y_1)

In [14]:
len(X_res)

13512

In [15]:

len(y_res)

13512

### Note:- since , i have not splited my data into train-test dataset so my independent and dependent data are in pandas dataframe so in order to implement in XGBoostingClassifier algo in pickled , model will not take dataframe data as input to predict , so i need to convert X(input data ) into numpy array , i m not converting y into numpy array¶


In [16]:
#type(X_res)
type(y_res)

pandas.core.frame.DataFrame

In [17]:
X=np.array(X_res)
y=np.array(y_res)

In [18]:
#type(X)
#type(y)

In [19]:
import seaborn as sns
import xgboost as xgb
# Classification metrices
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report, precision_score,recall_score,f1_score

In [20]:
xgb.__version__

'1.5.0'

### functionalizing the model

In [21]:

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import pickle

In [22]:
# Functionalize model fittting

def FitModel(X,Y,algo_name,algorithm,gridSearchParams,cv):
    np.random.seed(10)
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2)
    
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1)
    
    
    grid_result = grid.fit(x_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(x_test)
    cm = confusion_matrix(y_test, pred)
   # metrics =grid_result.gr
    print(pred)
    pickle.dump(grid_result,open(algo_name,'wb'))
   
    print('Best Params :',best_params)
    print('Classification Report :',classification_report(y_test,pred))
    print('Accuracy Score : ' + str(accuracy_score(y_test,pred)))
    print('Confusion Matrix : \n', cm)

In [23]:
param ={
            'n_estimators': [50, 100, 500, 1000, 2000],
           
        }
FitModel(X,y,'finalized_model5.pkl',XGBClassifier(),param,cv=5)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\CG-DTE\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\CG-DTE\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[17:16:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[1 1 0 ... 1 1 0]
Best Params : {'n_estimators': 500}
Classification Report :               precision    recall  f1-score   support

           0       0.99      0.97      0.98      1361
           1       0.97      0.99      0.98      1342

    accuracy                           0.98      2703
   macro avg       0.98      0.98      0.98      2703
weighted avg       0.98      0.98      0.98      2703

Accuracy Score : 0.9785423603403626
Confusion Matrix : 
 [[1322   39]
 [  19 1323]]


### Reloading the Saved Model

In [24]:
loaded_model = pickle.load(open("finalized_model5.pkl","rb"))

In [25]:
pred = loaded_model.predict(np.array([[32.3510,0.249760,1.2479,0.214020,0.477060,1.45820,3.8772,0.662950,1.522500,348690.00]]))

In [26]:

print(pred)

[0]


In [27]:
def change (prediction):
    if prediction==1:
        return 'yes'
    else:
        return 'No'

In [28]:
print('some of the features of company:-')
print('[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = 32.3510')
print('(gross profit + extraordinary items + financial expenses) / total assets = 0.249760')
print('sales (n) / sales (n-1) = 1.2479')
print('profit on operating activities / total assets = 0.214020')
print('gross profit (in 3 years) / total assets = 0.477060')
print('profit on operating activities / financial expenses = 1.45820')
print('operating expenses / short-term liabilities = 3.8772')
print('(current assets - inventory - receivables) / short-term liabilities = 0.662950')
print('(current assets - inventory) / short-term liabilities = 1.522500')
print(' (sales - cost of products sold) / sales = 348690.00')
print('\n\n')

print('will company go for bankruptcy ? {}'.format(change(pred)))

some of the features of company:-
[(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365 = 32.3510
(gross profit + extraordinary items + financial expenses) / total assets = 0.249760
sales (n) / sales (n-1) = 1.2479
profit on operating activities / total assets = 0.214020
gross profit (in 3 years) / total assets = 0.477060
profit on operating activities / financial expenses = 1.45820
operating expenses / short-term liabilities = 3.8772
(current assets - inventory - receivables) / short-term liabilities = 0.662950
(current assets - inventory) / short-term liabilities = 1.522500
 (sales - cost of products sold) / sales = 348690.00



will company go for bankruptcy ? No
